## Tomcat Static default Web resources

Default tomcat static webpages is located at

/usr/local/tomcat/webapps/ROOT

which we don't need in our app so we need to

RUN rm -rf ./webapps/*

## copy our war file to tomcat

COPY target/fleetman-0.0.1-SNAPSHOT.war /usr/local/tomcat/webapps/ROOT.war


## Running our docker container with CMD

setting our java arguement with ENV JAVA_OPTS

ENV JAVA_OPTS="-Dspring.profiles.active=docker-demo"

CMD ["catalina.sh", "run"]

## Build War file with maven
Build the war file with

In [ ]:
%%bash
mvn package

Build the image

In [ ]:
%%bash
docker image build -t fleetman-webapp .

Run the image

In [ ]:
%%bash
docker container run -p 8080:8080 -it fleetman-webapp

## Spring Boot has Tomcat

Since spring boot already has tomcat, we don't have  to install tomcat, we can just package our application as a jar. And run in any container that has java and run the jar file.

To package as jar change the pom.xml to package your application as jar instead of war.

    <groupId>com.virtualpairprogrammers</groupId>
    <artifactId>fleetman</artifactId>
    <version>0.0.1-SNAPSHOT</version>
    <packaging>jar</packaging>

And Run

In [ ]:
%%bash
java -Dspring.profiles.active=development -jar $(pwd)/target/fleetman-0.0.1-SNAPSHOT.jar

# Microservice Philosophy for Docker
In docker it is best practice for each container to run a service, you shouldn't run multiple services in 1 container. Because it is easier to maintain and manage if 1 container runs a service. For example if you have 2 services in 1 container and one of the service died, it is hard to diagnose which service died and bring up the service that died. If you decided to restart the container the healthy service would be brought down too.

For example if our tomcat application needs a database, we shouldn't add mysql to the same container that runs the same database.

<img src="../../docker/media/TwoServiceOneContainer.png" height="50%" width="50%">

We should have multiple containers and each container provide a single service, and the service is determined by the last CMD or ENTRYPOINT line in our Dockerfile.
Container communicate with each other through a network.

<img src="../../docker/media/EachContainerSingleService.png" height="50%" width="50%">


## Working with Mysql
pull down the image

In [ ]:
%%bash
docker container run -e MYSQL_ROOT_PASSWORD=****** -e MYSWL_DATABASE=fleetman -d mysql:8.0.16

Run the container

In [ ]:
%%bash
docker container exec -it 85 bash

Run mysql inside the container

In [ ]:
%%bash
mysql -uroot -ppassword
show databases

# Networking in Docker
The scheme networking uses is to add the container name to the DNS service. 

A container want to network with another container, they can simply use the container name of the other container as the domain name so we don't need to deal with any IP addresses.

Networks that comes with docker, lets list them

In [ ]:
%%bash
docker network ls

network named *bridge* is the default network, it is the network all of our container is conneting too. It is four outbound traffic, that is why we are able to ping google.com or install packages from remote repositories in our containers.

But *bridge* does not allow container to container networking.

## Container to Container Networking
If we want to perform container to container networking we need to create our own network.

Lets create our own network called *my-network*

In [ ]:
%%bash
docker network create my-network

Now spin up instance of our container and connect them to our network

In [ ]:
%%bash
docker container run --network my-network --name database -e MYSQL_ROOT_PASSWORD=**** -e MYSQL_DATABASE=fleetman -d mysql:8.0.16
    
docker container run --network my-network --name fleetman-webapp -d -p 80:8080 fleetman-webapp

Lets go into our fleetman-webapp container and ping our database container

In [ ]:
%%bash
docker container exec -it fleetman-webapp bash

ping database

## Connecting our Tomcat app to Mysql database in another container
In application-docker-demo.properties files
replace 

In [ ]:
spring.datasource.url=jdbc:h2:file:./Database

With

In [ ]:
spring.datasource.url=jdbc:mysql://database:3306/fleetman
spring.datasource.username=root
spring.datasource.password=password

Rebuilt the war file for the fleetman app with maven

In [ ]:
%%bash
mvn package

Rebuild the image with the new war file

In [ ]:
%%bash
docker image build -t fleetman-webapp .

Not spin up the container running the tomcat app

In [6]:
%%bash
docker container run --network my-network --name fleetman-webapp -d -p 80:8080 --rm fleetman-webapp

UsageError: %%bash is a cell magic, but the cell body is empty.


Extra argument --rm means that if container is stopped it will be removed. 

Lets got into our database container and look at the tables

In [ ]:
%%bash
docker container exec -it database bash

mysql -uroot -ppassword

use fleetman;

show tables;

select * from vehicles;